In [7]:
import os
import shutil
import numpy as np
import pandas as pd
import cv2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

root = '/Users/baohezhang/codes/cs598jp-prj/'

file_paths = [
    root + 'training10_0.tfrecords',
    root + 'training10_1.tfrecords',
    root + 'training10_2.tfrecords',
    root + 'training10_3.tfrecords',
    root + 'training10_4.tfrecords',
]

class TFRecordExtractor:
    def __init__(self, file_paths):
        self.file_paths = [os.path.abspath(file_path) for file_path in file_paths]
        self.count = 0
        
    def get_images(self):
        dataset = tf.data.TFRecordDataset(self.file_paths)
        dataset = dataset.map(self._extract_fn)
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            image_data_list = []
            try:
                while True:
                    image_data = sess.run(next_element)
                    image_data_list.append(image_data)
            except:
                pass
            
            return image_data_list
        
    def _extract_fn(self, tfrecord):
        # Extract features using keys
        features = {
            'label': tf.FixedLenFeature([], tf.int64),
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }
        
        parsed_data = tf.parse_single_example(tfrecord, features=features)
        label = parsed_data['label']
        image = tf.decode_raw(parsed_data['image'], tf.uint8)
        image = tf.reshape(image, [299, 299, 1])
        
        # Convert gray to 3 channell, then rescale to 227
        image = tf.image.grayscale_to_rgb(image)
        image = tf.image.resize(image, [227, 227])
        
        return [image, label]
    
    def post_process_images(self):
        image_data_list = self.get_images()
        benign_count = 0
        malignant_count = 0
        total_count = 0
        
        type_dict = {
            0: 'normal',
            1: 'benign_calci',
            2: 'benign_mass',
            3: 'malignant_calci',
            4: 'malignant_mass',
        }
        
        for image_data in image_data_list:
            label = image_data[1]
                
            if label == 0:
                continue
                
            total_count += 1 
            _type = None
            
            benign_label = 0

            if label == 1 or label == 2:
                _type = 'benign'
                benign_count += 1
            
            if label == 3 or label == 4:
                _type = 'malignant'
                malignant_count += 1
                benign_label = 1
            
            file_name = 'images/' + type_dict[label] + \
                        '/' + type_dict[label] + '_' + str(total_count) + '.png'
            img = image_data[0]
#             cv2.imwrite(root + file_name, img)
            
            if label == 2 or label == 4:
                with open(root + 'images/train_labels.txt', 'a') as f:
                    f.write(file_name + ' ' + str(benign_label) + '\n')
            
        print('normal: ' + str(total_count - benign_count - malignant_count))
        print('benign: ' + str(benign_count))
        print('malignant: ' + str(malignant_count))

TFRecordExtractor(file_paths).post_process_images()

normal: 0
benign: 4014
malignant: 3275
